## Выбор фреймворка




Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.

In [12]:
%load_ext autoreload
%autoreload 2


In [1]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA = load_mock_inputs()
print(f"Ключи data: {list[str](DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description', 'citation', 'paper_url', 'paper_reference', 'creator', 'contributor', 'collection_date', 'upload_date', 'language', 'licence', 'url', 'original_data_url', 'minio_url', 'format', 'file_id', 'default_target_attribute', 'ignore_attribute', 'row_id_attribute', 'num_rows', 'num_features', 'num_classes', 'num_missing_values', 'quality', 'dataset_type', 'tags', 'status', 'visibility', 'extra_info', 'local_path']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [3]:
from mas_automl.code_agent.load_mocks import load_mock_inputs
from IPython.display import display, Markdown

DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA = load_mock_inputs()

# Pretty display in notebook
display(Markdown(f"""
### Loaded Mock Inputs

#### Data Analyze Keys
{', '.join(DATA_ANALYZE.keys())}

#### Metadata Keys
{', '.join(METADATA.keys())}

#### Registry Frameworks
{', '.join(REGISTRY.keys())}

{', '.join(FINAL_DATA.keys())}
"""))





### Loaded Mock Inputs

#### Data Analyze Keys
summary, priority, steps, example_pipeline_snippet, frameworks_recommended, rationale, estimated_complexity, confidence

#### Metadata Keys
dataset_id, name, version, version_label, description, citation, paper_url, paper_reference, creator, contributor, collection_date, upload_date, language, licence, url, original_data_url, minio_url, format, file_id, default_target_attribute, ignore_attribute, row_id_attribute, num_rows, num_features, num_classes, num_missing_values, quality, dataset_type, tags, status, visibility, extra_info, local_path

#### Registry Frameworks
AutoGluon, H2O AutoML, LightAutoML

manifest, validation_report, metafeatures, preprocessing_recipe, code_agent_recommendation, run_metadata


In [4]:
# DATA_ANALYZE, METADATA, REGISTRY
import pandas as pd

PATH_TO_CSV = FINAL_DATA["manifest"]["local_path"]
dataset_df = pd.read_csv(PATH_TO_CSV)

dataset_df.info(), dataset_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   int64  
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   int64  
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   int64  
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   int64  
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

(None,
   checking_status  duration                  credit_history   purpose  \
 0              <0         6  critical/other existing credit  radio/tv   
 1        0<=X<200        48                   existing paid  radio/tv   
 
    credit_amount    savings_status employment  installment_commitment  \
 0         1169.0  no known savings        >=7                       4   
 1         5951.0              <100     1<=X<4                       2   
 
       personal_status other_parties  ...  property_magnitude age  \
 0         male single          none  ...         real estate  67   
 1  female div/dep/mar          none  ...         real estate  22   
 
    other_payment_plans housing existing_credits      job num_dependents  \
 0                 none     own                2  skilled              1   
 1                 none     own                1  skilled              1   
 
    own_telephone foreign_worker class  
 0            yes            yes  good  
 1           none       

In [5]:
# --- Инициализация клиента ---
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig

client = LLMClient(LLMConfig())

# --- Вызов выбора фреймворка ---


{'api_key': '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'}


In [6]:
client.chat('say asdasfa')

'Asdasfa! How can I assist you today?'

## Выбор фреймворка

## Итеративная генерация и тестирование кода

Ниже показан процесс итеративной генерации кода с тестированием через execnet_gateway


In [7]:
# Итеративная генерация и тестирование кода
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code
from mas_automl.code_agent.execnet_gateway import PythonSandboxClient
from pathlib import Path
import pandas as pd

CSV_PATH = FINAL_DATA["manifest"]["local_path"]
OUTPUT_DIR = str(Path(CSV_PATH).parent / "predictions")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

framework = "scikit-learn"
max_iterations = 3
feedback = ""

print(f"CSV путь: {CSV_PATH}")
print(f"Директория для предиктов: {OUTPUT_DIR}\n")



CSV путь: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\openml_31_credit-g.csv
Директория для предиктов: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\predictions



In [7]:

for iteration in range(1, max_iterations + 1):
    print(f"{'='*60}")
    print(f"Итерация {iteration}/{max_iterations}")
    print(f"{'='*60}\n")
    
    # Генерируем код
    print("Генерация кода...")
    code = generate_code(framework, client, iteration, feedback, FINAL_DATA)
    print(f"Сгенерированный код:\n{code}\n")
    
    # Тестируем код
    print("Тестирование кода...")
    tests_passed, test_feedback, predict_path = evaluate_code(
        code, framework, CSV_PATH, OUTPUT_DIR, iteration, FINAL_DATA
    )
    
    print(f"Тесты пройдены: {tests_passed}")
    print(f"Обратная связь: {test_feedback}")
    if predict_path:
        print(f"Путь к предиктам: {predict_path}")
        # Загружаем и показываем предикты
        try:
            pred_df = pd.read_csv(predict_path)
            print(f"\nПервые 5 строк предиктов:")
            print(pred_df.head())
        except Exception as e:
            print(f"Ошибка загрузки предиктов: {e}")
    
    print("\n")
    
    if tests_passed:
        print("✅ Все тесты пройдены! Останавливаем итерации.")
        break
    else:
        feedback = test_feedback
        print("❌ Тесты не пройдены. Переходим к следующей итерации с учетом обратной связи.\n")

print(f"\n{'='*60}")
print("Итоговый результат:")
print(f"Итераций выполнено: {iteration}")
print(f"Тесты пройдены: {tests_passed}")
print(f"Путь к предиктам: {predict_path}")

Итерация 1/3

Генерация кода...


2025-11-14 15:23:35,785 [INFO] Creating new execnet gateway with python=c:\Users\User1\Desktop\ITMO_bootcamp\.venv\Scripts\python.exe
2025-11-14 15:23:35,859 [INFO] Gateway and channel created.


Сгенерированный код:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

def train_model(train_df: pd.DataFrame, test_df: pd.DataFrame, label: str):
    numeric_features = ['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']
    categorical_features = ['checking_status', 'credit_history', 'purpose', 'savings_status', 'employment', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']
    
    X_train = train_df.drop(columns=[label])
    y_train = train_df[label]
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(), cate

NameError: name 'result' is not defined

## Просмотр сохраненных предиктов

Если предикты были успешно сохранены, можно загрузить и проанализировать их


In [ ]:
# Просмотр сохраненных предиктов
from pathlib import Path
import pandas as pd

if predict_path and Path(predict_path).exists():
    pred_df = pd.read_csv(predict_path)
    print(f"Загружено предиктов: {len(pred_df)}")
    print(f"\nПервые 10 строк:")
    print(pred_df.head(10))
    print(f"\nСтатистика:")
    print(pred_df.describe())
    
    # Простая метрика точности
    if 'true_label' in pred_df.columns and 'prediction' in pred_df.columns:
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(pred_df['true_label'], pred_df['prediction'])
        print(f"\nТочность (accuracy): {accuracy:.4f}")
else:
    print("Предикты не найдены. Запустите предыдущую ячейку для генерации.")


# СТАРЬЕ

In [12]:
from __future__ import annotations

import json
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Tuple

if __package__ is None or __package__ == "":
    from mas_automl.code_agent.load_mocks import load_mock_inputs  # type: ignore
    from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig  # type: ignore
    from mas_automl.code_agent.execnet_gateway import PythonSandboxClient, SandboxResult  # type: ignore
else:
    from .load_mocks import load_mock_inputs
    from .openai_wraper import LLMClient, LLMConfig
    from .execnet_gateway import PythonSandboxClient, SandboxResult

DEFAULT_MAX_ITERATIONS = 3


@dataclass
class PipelineResult:
    framework: str
    reason: str
    code: str
    tests_passed: bool
    iterations: int
    feedback: str
    predict_path: str | None = None


def evaluate_code(
    code: str,
    framework: str,
    csv_path: str,
    CSV_PATH_TO_PREDICT: str,
    output_dir: str,
    iteration: int,
    final_data: Dict[str, Any] | None = None,
) -> Tuple[bool, str, str | None]:
    """
    Выполняет код в песочнице, тестирует функцию train_model и сохраняет предикты.
    Возвращает (tests_passed, feedback, predict_path).
    """
    sandbox = PythonSandboxClient.get()
    
    # Получаем информацию о датасете
    target_column = "class"
    if final_data:
        target_column = final_data.get("manifest", {}).get("target_column", "class")
    print("1 - start testing")
    # Подготавливаем код для выполнения
    test_code = f"""
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from automl import AutoML


# Загружаем данные
df = pd.read_csv(r'{csv_path}')
print(f"Загружено строк: {{len(df)}}")

# Генерированный код пользователя
{code}

# Выполняем функцию
try:
    df['pred'] = np.random.rand(len(df))
    df.to_csv(r'{CSV_PATH_TO_PREDICT}', index=False)
    # print(f"Предикты сохранены в: {CSV_PATH_TO_PREDICT}")
    
    # Простые проверки
    errors = []
   
    if errors and any(errors):
        raise ValueError("; ".join([e for e in errors if e]))
    
    result = {{"ok": True, "predict_path": "mock", "message": "Все проверки пройдены"}}
    
except Exception as e:
    import traceback
    result = {{"ok": False, "predict_path": None, "message": str(e), "traceback": traceback.format_exc()}}

# Выводим результат в формате JSON для парсинга
print("RESULT_START")
print(json.dumps(result, ensure_ascii=False))
print("RESULT_END")
"""
    
    # Выполняем код в песочнице
    result = sandbox.run(test_code)
    print("2 - end testing")
  
    if not result.ok:
        feedback = f"Ошибка выполнения: {result.stderr}\n{result.stdout}"
        return False, feedback, None
    print("3 - try gateway")

    # Парсим результат из stdout
    try:
        stdout = result.stdout
        # Ищем маркеры RESULT_START и RESULT_END
        if "RESULT_START" in stdout and "RESULT_END" in stdout:
            start_idx = stdout.find("RESULT_START") + len("RESULT_START")
            end_idx = stdout.find("RESULT_END")
            result_json = stdout[start_idx:end_idx].strip()
            result_dict = json.loads(result_json)
        else:
            # Fallback: пытаемся найти JSON в stdout
            import re
            json_match = re.search(r'\{[^{}]*"ok"[^{}]*\}', stdout)
            if json_match:
                result_dict = json.loads(json_match.group())
            else:
                result_dict = {"ok": False, "message": "Не удалось найти результат в stdout"}
    except Exception as e:
        feedback = f"Ошибка парсинга результата: {e}\nStdout: {result.stdout}\nStderr: {result.stderr}"
        return False, feedback, None
    print("4 - end gateway")

    if result_dict.get("ok", False):
        predict_path = result_dict.get("predict_path")
        message = result_dict.get("message", "Проверки пройдены")
        return True, message, predict_path
    else:
        error_msg = result_dict.get("message", "Неизвестная ошибка")
        traceback_info = result_dict.get("traceback", "")
        feedback = f"Тесты не пройдены: {error_msg}\n{traceback_info}"
        return False, feedback, None



In [9]:
CSV_PATH_TO_PREDICT = 'C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\TEST\\test1__.csv'

In [14]:
# print("Генерация кода...")
# code = generate_code(framework, client, iteration, feedback, FINAL_DATA)
# print(f"Сгенерированный код:\n{code}\n")
iteration = 1 
# Тестируем код
print("Тестирование кода...")
tests_passed, test_feedback, predict_path = evaluate_code(
    "print(5)", framework, CSV_PATH, CSV_PATH_TO_PREDICT, OUTPUT_DIR, iteration, FINAL_DATA
)

predict_path = CSV_PATH_TO_PREDICT

print(f"Тесты пройдены: {tests_passed}")
print(f"Обратная связь: {test_feedback}")
if predict_path:
    print(f"Путь к предиктам: {predict_path}")
    # Загружаем и показываем предикты
    try:
        pred_df = pd.read_csv(predict_path)
        print(f"\nПервые 5 строк предиктов:")
        print(pred_df.head())
    except Exception as e:
        print(f"Ошибка загрузки предиктов: {e}")

print("\n")

2025-11-14 19:04:38,553 [INFO] Creating new execnet gateway with python=c:\Users\User1\Desktop\ITMO_bootcamp\.venv\Scripts\python.exe
2025-11-14 19:04:38,645 [INFO] Gateway and channel created.
2025-11-14 19:04:38,646 [DEBUG] Sending code to sandbox (attempt 1):
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from automl import AutoML


# Загружаем данные
df = pd.read_csv(r'C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\openml_31_credit-g.csv')
print(f"Загружено строк: {len(df)}")

# Генерированный код пользователя
print(5)

# Выполняем функцию
try:
    df['pred'] = np.random.rand(len(df))
    df.to_csv(r'C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\TEST\test1__.csv', index=False)
    # print(f"Предикты сохранены в: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\TEST\test1__.csv")

    # Простые проверки
    errors = []

    if errors and any(errors):
        raise ValueError("; ".join([e for e in errors if e]))

 

Тестирование кода...
1 - start testing


2025-11-14 19:04:44,692 [DEBUG] Received result from sandbox (attempt 1).
2025-11-14 19:04:44,693 [INFO] [sandbox stdout]
Загружено строк: 1000
5
RESULT_START
{"ok": true, "predict_path": "mock", "message": "Все проверки пройдены"}
RESULT_END


2 - end testing
3 - try gateway
4 - end gateway
Тесты пройдены: True
Обратная связь: Все проверки пройдены
Путь к предиктам: C:\Users\User1\Desktop\ITMO_bootcamp\data\datasets\TEST\test1__.csv

Первые 5 строк предиктов:
  checking_status  duration                  credit_history  \
0              <0         6  critical/other existing credit   
1        0<=X<200        48                   existing paid   
2     no checking        12  critical/other existing credit   
3              <0        42                   existing paid   
4              <0        24              delayed previously   

               purpose  credit_amount    savings_status employment  \
0             radio/tv         1169.0  no known savings        >=7   
1             radio/tv         5951.0              <100     1<=X<4   
2            education         2096.0              <100     4<=X<7   
3  furniture/equipment         7882.0              <100     4<=X<7   
4              new car         4870.0              

In [ ]:
from IPython.display import display, Markdown

# Your existing call (unchanged)
from mas_automl.code_agent.base_pipeline import choose_framework

framework, reason, prompt = choose_framework(
    DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA, client)

# Pretty display in notebook
display(Markdown(f"""
### Selected Framework
**Framework:** {framework}

### Reasoning
{reason}

### Generated Prompt
{prompt}
"""))



### Selected Framework
**Framework:** H2O AutoML

### Reasoning
H2O AutoML является наиболее стабильным и универсальным выбором для табличных данных данного размера (1000 строк, 21 признаков) и типа задачи (бинарная классификация). Оно предлагает надежный стекинг моделей (GBM, XGBoost, GLM, DRF, DeepLearning), автоматическую кросс-валидацию, раннюю остановку и поддержку production-сценариев. Несмотря на отсутствие поддержки мультимодальности и GPU для всех моделей, оно хорошо подходит для задач классификации с высокой воспроизводимостью и стабильностью, что особенно важно в корпоративных и промышленных условиях. В отличие от AutoGluon, которое требует больше ресурсов и может быть менее предсказуемо на небольших наборах данных, H2O AutoML обеспечивает более предсказуемое поведение и более широкую совместимость с существующими инфраструктурными решениями.

### Generated Prompt
input_variables=['analysis_json', 'frameworks_list', 'metadata_json'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты — эксперт по AutoML и ML-инженер. Твоя задача — выбрать наиболее подходящий AutoML-фреймворк для табличных данных. Оцени качество данных, размер, тип задачи, ограничения и предложи лучший вариант из списка.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_json', 'frameworks_list', 'metadata_json'], input_types={}, partial_variables={}, template='Вот анализ датасета и метаданные:\n\n### 📊 Data Analysis\n{analysis_json}\n\n### 🧾 Metadata\n{metadata_json}\n\n### ⚙️ Доступные AutoML фреймворки\n{frameworks_list}\n\nПоясни свой выбор кратко, но содержательно. Если несколько подходят, выбери наиболее универсальный и стабильный вариант. Ответ верни в формате JSON:\n{{"framework": "...", "reason": "..."}}'), additional_kwargs={})]


In [7]:
print(type(prompt))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


# Кодогенерация

In [8]:
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code

code_sample = generate_code(framework, client, iteration=1, feedback="")
print(code_sample)
print(evaluate_code(code_sample, framework))


import h2o
from h2o.automl import H2OAutoML

def train_model(train_df, test_df, label):
    h2o.init()
    train = h2o.H2OFrame(train_df)
    test = h2o.H2OFrame(test_df)
    train[label] = train[label].asfactor()
    test[label] = test[label].asfactor()
    
    aml = H2OAutoML(max_models=10, seed=1)
    aml.train(x=train.columns, y=label, training_frame=train)
    
    return aml.leader
(True, 'Проверки пройдены.')


In [47]:
import sklearn; sklearn.__version__


'1.7.2'


Usage:   
  pip freeze [options]

no such option: -m


In [1]:
from automl import AutoML